# Playback OpenSim Mot file
In this notebook is shown how to load a mot file generate by OpenSim and play it back on the MyoLeg model.

In [1]:
import os
import time
from urllib.parse import urlparse

import mujoco
import numpy as np
import pandas as pd
import requests
import scipy
import skvideo.io

def read_mot(filename):
    # --- Save the file ---
    filename = 'subject01_walk1_ik.mot'
    print(filename)
    if os.path.exists(filename):
        print(f"File '{filename}' already exists.")
    else:
        try:
            req = requests.get(url, allow_redirects=True, stream=True)
            print('yes')
            req.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

            parsed_url = urlparse(url)
            print(parsed_url)
            filename = os.path.basename(parsed_url.path)
            print(filename)
            if not filename:  # Handle cases where the path doesn't include a filename
                filename = "downloaded_file"  # Provide a default name
            print(f"File '{filename}' downloaded successfully.")
        except requests.exceptions.RequestException as e:
            print(f"Error downloading file: {e}")
    # Initialize a counter for the number of rows to skip
    skiprows = 0

    # Open the file and read line by line
    with open(filename, "r", encoding="utf-8") as file:
        for line in file:
            # Increment the skiprows counter for each line until "endheader" is found
            if line.strip() == "endheader":
                break
            skiprows += 1

    # Now read the CSV file, skipping the determined number of rows
    df = pd.read_csv(filename, sep="\t", skiprows=skiprows + 1)

    return df



In [2]:
def read_mat(mat_file_path):
    """
    Reads a MATLAB .mat file containing IK data and returns a labeled DataFrame.
    Handles different MATLAB structure formats.
    """
    try:
        mat_data = scipy.io.loadmat(mat_file_path)
        
        # Method 1: Direct access if not nested
        if 'IKAngData' in mat_data and 'IKAngDataLabel' in mat_data:
            data = mat_data['IKAngData']
            labels = mat_data['IKAngDataLabel']
        
        # Method 2: Nested structure access
        else:
            # Navigate through the nested structure
            datastr = mat_data['Datastr'][0,0]
            resample = datastr['Resample'][0,0]
            sych = resample['Sych'][0,0]
            
            data = sych['IKAngData']
            labels = sych['IKAngDataLabel']
            
            # Handle 2D label array
            if labels.ndim == 2:
                labels = labels[0]
            
        # Convert labels to Python strings
        column_labels = [str(label[0]) if isinstance(label, np.ndarray) else str(label) 
                        for label in labels.flatten()]
        
        return pd.DataFrame(data, columns=column_labels)
        
    except Exception as e:
        print(f"Error reading MAT file: {e}")
        raise


In [3]:
df_mat = read_mat('Subj04_jump.mat')
df_run = read_mat('Subj04_run_81.mat')

In [4]:
'''
df_mat.rename(columns={
    'lumbar_extension': 'L5_S1_Flex_Ext',
    'lumbar_bending': 'L5_S1_Lat_Bending', 
    'lumbar_rotation': 'L5_S1_axial_rotation'
}, inplace=True)

df_run.rename(columns={
    'lumbar_extension': 'L5_S1_Flex_Ext',
    'lumbar_bending': 'L5_S1_Lat_Bending', 
    'lumbar_rotation': 'L5_S1_axial_rotation'
}, inplace=True)
'''

"\ndf_mat.rename(columns={\n    'lumbar_extension': 'L5_S1_Flex_Ext',\n    'lumbar_bending': 'L5_S1_Lat_Bending', \n    'lumbar_rotation': 'L5_S1_axial_rotation'\n}, inplace=True)\n\ndf_run.rename(columns={\n    'lumbar_extension': 'L5_S1_Flex_Ext',\n    'lumbar_bending': 'L5_S1_Lat_Bending', \n    'lumbar_rotation': 'L5_S1_axial_rotation'\n}, inplace=True)\n"

In [5]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 400):

  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")


In [6]:
url = "https://raw.githubusercontent.com/opensim-org/opensim-models/refs/heads/master/Pipelines/Gait2392_Simbody/OutputReference/subject01_walk1_ik.mot"
df_mot = read_mot(url)

mj_model = mujoco.MjModel.from_xml_path(
    "../../../myosuite/simhive/myo_sim/leg/myolegs_abdomen.xml"
)
mj_data = mujoco.MjData(mj_model)

joint_names = [mj_model.joint(jn).name for jn in range(mj_model.njnt)]
print(joint_names)

def output_frame(df):
    subc = [c for c in df.columns if c in joint_names]

    print(
        f"Joints in the Mot files that are not present in the MJC model: {set(subc) - set(joint_names)}"
    )

    # ---- camera settings
    camera = mujoco.MjvCamera()
    camera.azimuth = 120
    camera.distance = 3
    camera.elevation = -15.0
    camera.lookat = [0,0,.75]
    options_ref = mujoco.MjvOption()
    options_ref.flags[:] = 0
    options_ref.geomgroup[1:] = 0
    renderer_ref = mujoco.Renderer(mj_model)
    renderer_ref.scene.flags[:] = 0
    frames=[]
    from tqdm import tqdm
    for t in tqdm(range(len(df)), desc="Rendering frames"):
        for jn in subc:
            mjc_j_idx = mj_model.joint(joint_names.index(jn)).qposadr
            mj_data.qpos[mjc_j_idx] = np.deg2rad(df[jn].loc[t])
            if "knee_angle" in jn:  # knee joints have negative sign in myosuite
                mj_data.qpos[mjc_j_idx] *= -1

        mujoco.mj_forward(mj_model, mj_data)
        renderer_ref.update_scene(mj_data, camera=camera)#, scene_option=options_ref)
        frame = renderer_ref.render()
        frames.append(frame)
    
    return frames

mot_frames = output_frame(df_mot)

os.makedirs('videos', exist_ok = True)
output_name = 'videos/playback_mot.mp4'
skvideo.io.vwrite(output_name, np.asarray(mot_frames),outputdict={"-pix_fmt": "yuv420p"})
# show in the notebook
show_video('videos/playback_mot.mp4')

subject01_walk1_ik.mot
File 'subject01_walk1_ik.mot' already exists.
['root', 'lumbar_extension', 'lumbar_bending', 'lumbar_rotation', 'hip_flexion_r', 'hip_adduction_r', 'hip_rotation_r', 'knee_angle_r_translation2', 'knee_angle_r_translation1', 'knee_angle_r', 'knee_angle_r_rotation2', 'knee_angle_r_rotation3', 'ankle_angle_r', 'subtalar_angle_r', 'mtp_angle_r', 'knee_angle_r_beta_translation2', 'knee_angle_r_beta_translation1', 'knee_angle_r_beta_rotation1', 'hip_flexion_l', 'hip_adduction_l', 'hip_rotation_l', 'knee_angle_l_translation2', 'knee_angle_l_translation1', 'knee_angle_l', 'knee_angle_l_rotation2', 'knee_angle_l_rotation3', 'ankle_angle_l', 'subtalar_angle_l', 'mtp_angle_l', 'knee_angle_l_beta_translation2', 'knee_angle_l_beta_translation1', 'knee_angle_l_beta_rotation1']
Joints in the Mot files that are not present in the MJC model: set()


Rendering frames: 100%|██████████| 211/211 [00:00<00:00, 646.30it/s]


In [7]:
mat_frames = output_frame(df_mat)
output_name = 'videos/playback_mat.mp4'
skvideo.io.vwrite(output_name, np.asarray(mat_frames),inputdict = {"-r":'100'}, outputdict={"-pix_fmt": "yuv420p"})
# show in the notebook
show_video('videos/playback_mat.mp4')

Joints in the Mot files that are not present in the MJC model: set()


Rendering frames: 100%|██████████| 3584/3584 [00:05<00:00, 677.80it/s]


In [8]:
run_frames = output_frame(df_run)
output_name = 'videos/playback_run.mp4'
skvideo.io.vwrite(output_name, np.asarray(run_frames),inputdict = {"-r":'100'}, outputdict={"-pix_fmt": "yuv420p"})
# show in the notebook
show_video('videos/playback_run.mp4')

Joints in the Mot files that are not present in the MJC model: set()


Rendering frames: 100%|██████████| 6553/6553 [00:08<00:00, 752.63it/s]
